* comment mixer les features tf-idf et les autres, pour leur donner du poids? Lire des articles,
* faire deux modèles et mixer
* comment gérer l'ironie?
* Faire une fonction qui va uniquement prédire sur X derniers tweets de la collection tweet

In [146]:
import pymongo as pym
import nltk.data
import re
import string
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
import stop_words
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
import time

In [124]:
stops = set(['rt','ds','qd','ss','ns','vs','nn','amp','gt','gd','gds','tt','pr','ac','mm', 'qu',
            '``', 'ni', 'ca', 'le', 'les', ' ', 'si', '$', '^', 'via', 'ils','pour','une','que','quel'] +
            list('@ن%£€‘:&;') + list('abcdefghijklmnopqrstuvwxyz')+
            stop_words.get_stop_words(language='fr')+stopwords.words('french'))
for e in ['ne','pas','sans'] :
    stops.remove(e)


In [134]:
def tweetPreprocessing(collection, nLastTweets=0, retweet=False):

    tweets = collection.find(filter={'text':{'$exists':True}}, 
                             projection={'_id':False}).sort('$natural',-1).limit(nLastTweets)  
    df = pd.DataFrame()
    listTweets, listCandidats, listSentiments = [], [], []

    for t in tweets: 
        if not retweet: # filtrage des retweets
            if 'rt @' in t['text']:
                continue
        # comptes
        a = t['text'].count('!')
        b = t['text'].count('?')
        c = t['text'].count('#')
        d = t['text'].count('"')
        e = t['text'].count('http')
        f = t['text'].count(':')
        g = t['text'].count('»')
        h = t['text'].count('@')
        
        # mot tronqué
        t['text'] = re.sub(r'\w*…', '', t['text'])
        
        # caracteres speciaux
        t['text'] = re.sub(r'\xad', '-',
                           re.sub(r'\n', ' ', # retrait des sauts de ligne
                       #           re.sub(r'\[a-zA-Z]*(?!\S)', '', # retrait de ce qui n'est pas un mot
                                         re.sub(r'(?:htt)\S*', '', # retrait des liens http
                                                re.sub(r'^rt.*: ', '', # retrait de la mention retweet
                                                       re.sub(r'\d', '', # retrait des chiffres
                                                              re.sub(r',;!?\/\*(){}«»', ' ', t['text']))))))
        
        t['text'] = re.sub('|'.join(['’', '_', '/', '-', '\'', '“', '\.']), ' ', t['text'])
        
        # accents (il faut laisser ce bloc)
#         t['text'] = re.sub('|'.join('Ééèêë'), 'e', t['text'])
#         t['text'] = re.sub('|'.join('àâä'), 'a', t['text'])
#         t['text'] = re.sub('|'.join('ç'), 'c', t['text'])
#         t['text'] = re.sub('|'.join('œ'), 'oe', t['text'])
#         t['text'] = re.sub('|'.join('Ôôö'), 'o', t['text'])
#         t['text'] = re.sub('|'.join('îï'), 'i', t['text'])
#         t['text'] = re.sub('|'.join('ùû'), 'u', t['text'])
        
        # apostrophes
        t['text'] = re.sub('|'.join([elem + '\'' for elem in 'cdjlmnst']), '', t['text'])
        
        tokenizer = TreebankWordTokenizer()
        t['text'] = tokenizer.tokenize(t['text'])
        t['text'] = [token for token in t['text'] if (token not in stops) and (len(token)>2)]
        negation = 0
        for token in t['text'] : 
            if token in ['ne','pas','n\'','n','sans'] : 
                negation = 1
        while '' in t['text']:
            t['text'].pop('')
            
        if t['text']: # test si liste non vide
            listTweets.append(t['text'])
            try:
                listCandidats.append(t['candidat'])
            except:
                listCandidats.append(None)
            
            try:
                listSentiments.append(t['sentiment'])
            except:
                listSentiments.append(None)
                
            rec = pd.DataFrame([[a, b, c, d, e, f,g, h, negation]], 
                               columns=['!', '?', '#', '"', '_http_',':','»','@','négation'])
            df = df.append(rec, ignore_index=True)
        
    df['text'], df['candidat'], df['sentiment'] = listTweets, listCandidats, listSentiments
    return df

def build_feat_mat(df_tweets):
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                use_idf=True, norm='l1', binary=False, min_df=.0001, max_df=0.6)
    X = vectorizer.fit_transform(df_tweets['text'].apply(' '.join))
    hstack((X, df_tweets[['!', '?', '#', '"', '_http_',':','»']]))

    return X

def getSentiments(client, n_predict, retweet, full_retrain=True) : 
    if full_retrain :
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 0, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 0, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        n_samples, vocabulaire = X.shape
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))
        params = {'penalty':['l2'], 'C' : np.logspace(-2,2,100), 
                  'solver':['lbfgs'], 'multi_class':['ovr','multinomial']}
        lr = LogisticRegression(max_iter=500)
        GCV = GridSearchCV(lr, params, verbose=1, n_jobs=-1)
        GCV.fit(X[:-n_predict], y[:-n_predict])
        print(GCV.best_params_)
        model = LogisticRegression(penalty=GCV.best_params_['penalty'],
                                   C=GCV.best_params_['C'], 
                                   solver=GCV.best_params_['solver'], multi_class=GCV.best_params_['multi_class'])   
    
    else :
        f = open('sentiment_model.pkl','rb')
        model = pickle.load(f)
        f.close()
        # bases utilisées
        df = tweetPreprocessing(client.tweet.train, 500, retweet)
        df2 = tweetPreprocessing(client.tweet.labelised, 500, retweet)
        try:
            df = pd.concat([df, df2], axis=0, ignore_index=True)
        except:
            pass
        df = df.sample(frac=1.0, replace=False) # mélange des lignes
        X = build_feat_mat(df)
        y = df['sentiment']
        n_samples, vocabulaire = X.shape
        print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))

    model.fit(X[:-n_predict], y[:-n_predict])
    predictions = model.predict(X[n_samples - n_predict:])
    print('Score', np.sum(predictions == y[n_samples - n_predict:]) / len(predictions)) 
    f = open('sentiment_model.pkl','wb')
    pickle.dump(model, f)
    f.close()
    print('Model saved')
    return predictions

def cleanTrainDb(collection):
    textCleanPipeline = [{"$group":{"_id":"$text", "dups":{"$push":"$_id"},"count":{"$sum":1}}},{"$match":{"count":{"$gt":1}}}]
    duplicates = []
    count = 0
    try:
        for doc in collection.aggregate(textCleanPipeline) :
            it = iter(doc['dups'])
            next(it)
            for id in it:
                count += 1
                duplicates.append(pym.DeleteOne({'_id':id}))
        if duplicates:
            collection.bulk_write(duplicates)
    except:
        pass
    
    print(count, 'doublons retirés.')
    client.close()

In [135]:
client = pym.MongoClient('localhost', 27018)
#client = pym.MongoClient('localhost', 27017)
collection = client.tweet.train
collection2 = client.tweet.labelised
print(collection.count())
print(collection2.count())

6246
1421


### Test du nettoyage des tweets

In [ ]:
a = tweetPreprocessing(collection, retweet=True)

In [ ]:
for a in a['text'][:20]:
    print(a)

### Test du nettoyage doublons

In [128]:
cleanTrainDb(collection)
cleanTrainDb(collection2)

0 doublons retirés.
0 doublons retirés.


## Train et test du modèle

In [136]:
a = getSentiments(client, 600, retweet=True, full_retrain=True)
# attention en cas de retrain périodique on ne prend que les 500 derniers tweets, il faut donc prédire sur moins de 500
print(len(a[a==1]), len(a[a==0]))

Tweets : 7635 / Mots : 12540
Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  4.6min finished


{'C': 17.07352647470692, 'multi_class': 'ovr', 'solver': 'lbfgs', 'penalty': 'l2'}
Score 0.636666666667
Model saved
135 114


# Essai sur un Xgboost

In [175]:
def MAPE(preds, dtrain):
    """
    This is the function used by XGBoost as metric.
    """
    labels = dtrain.get_label()
    return "MAPE", np.mean(np.abs((labels - preds) / labels)) * 100

def build_feat_mat(df_tweets):
    vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', decode_error='strict',
                                use_idf=True, norm='l2', binary=False, min_df=.0005, max_df=0.5)
    tfidf = vectorizer.fit_transform(df_tweets['text'].apply(' '.join))
    X = pd.DataFrame(tfidf.toarray())
    X[['!', '?', '#', '"', '_http_',':','»','@','négation']] = df_tweets[['!', '?', '#', '"', '_http_',':','»','@','négation']]
    return X


def TrainXGBoostModel(client, retweet=True, gpuEnabled=False):
    df = tweetPreprocessing(client.tweet.train, 0, retweet)
    df2 = tweetPreprocessing(client.tweet.labelised, 0, retweet)
    try:
        df = pd.concat([df, df2], axis=0, ignore_index=True)
    except:
        pass
    df = df.sample(frac=1.0, replace=False) # mélange des lignes
    X = build_feat_mat(df)
    df[df['sentiment'] == -1] = 2
    y = df['sentiment']
    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2) # 20 % for validation
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2) # 20 % for test
    n_samples, vocabulaire = X.shape
    print('Tweets : ' + str(n_samples) + ' / ' + 'Mots : ' + str(vocabulaire))

    boostRounds = 10000
    # XGBoost data structure
    xgtrain = xgb.DMatrix(x_train.values, y_train.values)
    xgeval = xgb.DMatrix(x_val.values, y_val.values)
    xgtest = xgb.DMatrix(x_test.values)
     
    
    # Generate the watchlist
    watchlist = [(xgtrain, "train"), (xgeval, "eval")]
    print("Train the XGBoost model...")
    startTime = time.time()
    param_test1 = {'max_depth':range(3,10,1), 'min_child_weight':range(1,6,1)}
    
    gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5, 
    min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
    objective= 'multi:softmax', nthread=3, scale_pos_weight=1, seed=27), 
    param_grid = param_test1, scoring='roc_auc',n_jobs=3,iid=False, cv=5, verbose=9)
    
    gsearch1.fit(x_train[:500], y_train[:500])
    clf = xgb.train(param_test1, xgtrain, boostRounds, evals = watchlist, maximize = True, verbose_eval = 100)

    #Make predictions
    print("Make predictions for testing set...")
    predictions_sr = pd.Series(clf.predict(xgtest, ntree_limit = clf.best_iteration), index = x_test.index)
    return predictions_sr, clf, y_test



In [ ]:
preds, model, truth = TrainXGBoostModel(client)
print('Score', np.sum(truth  == preds) / len(preds)) 

Tweets : 7635 / Mots : 2568
Train the XGBoost model...
Fitting 5 folds for each of 35 candidates, totalling 175 fits
[CV] min_child_weight=1, max_depth=3 .................................
[CV] min_child_weight=1, max_depth=3 .................................
[CV] min_child_weight=1, max_depth=3 .................................
